In [ ]:
from module.graph_loader import load_graph_from_place
from module.line_extension import linestring_extension, tunnel_linestring_extension
from module.split_road import split_road_and_change_nodes_state
from module.generator_node_edge import *
from module.cluster import * 
from module.cluster_spread import *
from module.make_map import *

from shapely.geometry import MultiLineString, Point
from shapely.ops import linemerge
import pandas as pd
import geopandas as gpd
import pydeck as pdk
import numpy as np
import itertools
import pickle
from tqdm import tqdm
import copy
import os 
import warnings 

warnings.filterwarnings('ignore')

In [ ]:
# HJD = gpd.read_file('./data/extra_data/HangJeongDong_ver20220401.geojson')
# main_city = HJD[['sido', 'sidonm']].drop_duplicates().reset_index(drop=True)
# main_city = main_city.loc[~main_city['sidonm'].isin(['경기도', '강원도', '충청북도', '충청남도', '전라북도', '전라남도', '경상북도', '경상남도'])]

# main_city_dict = {row['sidonm'][:2]: row['sido'] for _, row in main_city.iterrows()}

In [ ]:
HJD = gpd.read_file('./data/extra_data/HangJeongDong_ver20220401.geojson')
minor_city = HJD.loc[HJD['sidonm'].isin(['경기도', '강원도', '충청북도', '충청남도', '전라북도', '전라남도', '경상북도', '경상남도'])]

minor_city = minor_city[['adm_nm', 'adm_cd']]
minor_city['adm_nm'] = [' '.join(i.split(' ')[:2]) for i in minor_city['adm_nm']]
minor_city = minor_city.loc[~minor_city['adm_nm'].duplicated()]
minor_city = minor_city.reset_index(drop=True)

minor_dict = {'경기도 고양시덕양구': '경기도 고양시 덕양구',
             '경기도 성남시분당구': '경기도 성남시 분당구',
             '경기도 안산시상록구': '경기도 안산시 상록구',
             '경기도 고양시일산동구': '경기도 고양시 일산동구',
             '경기도 용인시수지구': '경기도 용인시 수지구',
             '경기도 수원시영통구': '경기도 수원시 영통구',
             '경기도 안산시단원구': '경기도 안산시 단원구',
             '경기도 수원시팔달구':'경기도 수원시 팔달구',
             '경상남도 창원시마산회원구': '경상남도 창원시 마산회원구',
             '경기도 안양시만안구': '경기도 안양시 만안구',
             '경상남도 창원시의창구': '경상남도 창원시 의창구',
             '경기도 용인시처인구':'경기도 용인시 처인구',
             '전라북도 전주시덕진구': '전라북도 전주시 덕진구',
             '경상남도 창원시성산구': '경상남도 창원시 성산구',
             '경상남도 창원시마산합포구':'경상남도 창원시 마산합포구',
             '충청북도 청주시청원구': '충청북도 청주시 청원구',
             '경기도 성남시수정구': '경기도 성남시 수정구',
             '경기도 용인시기흥구': '경기도 용인시 기흥구',
             '경기도 고양시일산서구': '경기도 고양시 일산서구',
             '경기도 안양시동안구': '경기도 안양시 동안구',
             '충청북도 청주시서원구': '충청북도 청주시 서원구',
             '경기도 성남시중원구': '경기도 성남시 중원구',
             '충청남도 천안시동남구': '충청남도 천안시 동남구',
             '경상북도 포항시북구': '경상북도 포항시 북구',
             '충청남도 천안시서북구': '충청남도 천안시 서북구',
             '충청북도 청주시흥덕구': '충청북도 청주시 흥덕구',
             '경기도 수원시장안구': '경기도 수원시 장안구',
             '경기도 수원시권선구': '경기도 수원시 권선구',
             '충청북도 청주시상당구': '충청북도 청주시 상당구', 
             '경상북도 포항시남구': '경상북도 포항시 남구',
             '전라북도 전주시완산구': '전라북도 전주시 완산구',
             '경상남도 창원시진해구': '경상남도 창원시 진해구'}

minor_city['adm_nm'] = [minor_dict[i] if i in minor_dict.keys() else i for i in minor_city['adm_nm']]
minor_city['adm_cd'] = [i[:5] for i in minor_city['adm_cd']]

In [ ]:
# minor_city = minor_city.loc[~minor_city['adm_cd'].isin(os.listdir('./data/result_data/'))]
# minor_city = minor_city.tail(80)

In [ ]:
for _,i in tqdm(minor_city.iterrows()):

    place = i['adm_nm']
    region_code = i['adm_cd']
    
    if region_code in os.listdir('./data/result_data/'):
        continue
    
    '''
    Step 1. OSMNX을 통한 graph data 로드 
    '''
    # G1 : 베이스 도시도로, G2 : 자세한 터널을 추가하기 위한 도시도로
    G1, G2 = load_graph_from_place(place, region_code)

    # 도시도로 추출의 위해 고속도로를 제거했기 때문에 고속도로로 인해 나눠진 도로 다시 연결
    road_nodes, road_edges = linestring_extension(G1)
    tunnel_nodes, tunnel_edges = tunnel_linestring_extension(G2)

    '''
    Step 2. main road(간선도로), minor road(집산도로) 분리 후 전처리
    '''
    # main road-간선도로(주간선, 간선, 보조간선), minor road-집산도로로 분리
    main_road, non_main_road = split_road_and_change_nodes_state([road_nodes, road_edges], [tunnel_nodes, tunnel_edges])

    ## 자세한 터널 위치를 추가하고 nodes, edges 업데이트 
    main_road, non_main_road = generate_tunnel_edges(main_road, non_main_road)

    # 터널 양방향으로 나눠진 경우 하나의 터널로 군집
    try:
        main_road_nodes, main_road_edges = cluster_for_tunnel_1(main_road[0], main_road[1])              # 간선도로 
    except:
        main_road_nodes, main_road_edges = main_road
    try:
        non_main_road_nodes, non_main_road_edges = cluster_for_tunnel_1(non_main_road[0], non_main_road[1])  # 집산도로 
    except:
        non_main_road_nodes, non_main_road_edges = non_main_road

    # 간선도로 - 도로 연장 터널 22, 교차로 터널 분류 21
    try:
        main_road_nodes, main_road_edges = cluster_for_tunnel_2(main_road_nodes, main_road_edges)
    except:
        pass
    '''
    Step 3. 복잡한 교차로 군집화
    '''
    # 간선도로 하나의 교차로에 복잡한 노드가 있는 교차로 하나로 인식하기 위해 군집화
    main_road_nodes, main_road_edges = cluster_for_cross(main_road_nodes, main_road_edges)
    
    '''
    Step 4. 도로 분할
    '''
    ### 간선도로 (도로 교통법에 따른 교차로 150m, 단일로 600m)
    # cross segement (meter 150) 
    main_road_nodes, main_road_edges = generate_cross_nodes_edeges(main_road_nodes, main_road_edges)   
    # single way segment (meter 600) 
    main_road_nodes, main_road_edges = generate_singleway_nodes_edeges(main_road_nodes, main_road_edges)
    
    # cluster fillna
    main_road_nodes.cluster = main_road_nodes.cluster.fillna(9999)
    main_road_edges.cluster = main_road_edges.cluster.fillna(9999)
    # tunnel fillna 
    main_road_edges.tunnel = main_road_edges.tunnel.fillna(0)
    
    # 간선도로 - 150m 공간적 범위 정의
    main_road_nodes, main_road_edges = generate_main_road_cluster(main_road_nodes, main_road_edges)
    
    
    ### 집산도로 (간선도로로 나눠진 집산도로를 블럭단위로 군집화)
    non_main_road_nodes, non_main_road_edges = generate_non_main_road_cluster(non_main_road_nodes, non_main_road_edges)
    try:
        non_main_road_edges.cluster = non_main_road_edges.cluster.fillna(-1)
    except:
        non_main_road_edges['cluster'] = -1
    
    # cross segement (meter 150)
    non_main_road_nodes, non_main_road_edges = generate_cross_nodes_edeges(non_main_road_nodes, non_main_road_edges)   
    # single way segment (meter 600) 
    non_main_road_nodes, non_main_road_edges = generate_singleway_nodes_edeges(non_main_road_nodes, non_main_road_edges) 

    non_main_road_nodes.connect_inf = non_main_road_nodes.connect_inf.fillna(0)
    try:
        non_main_road_nodes.cluster = non_main_road_nodes.cluster.fillna(-1)
    except:
        non_main_road_nodes['cluster'] = -1
    '''
    Step 5. raw data 저장
    '''
    ### 미사용 columns 삭제
    # edges
    main_road_edges = main_road_edges[['u', 'v', 'key', 'highway', 'length', 'geometry', 'tunnel', 'cluster']]
    non_main_road_edges = non_main_road_edges[['u', 'v', 'key', 'highway', 'length', 'geometry', 'tunnel', 'cluster', 'M_category']]
    # nodes
    main_road_nodes = main_road_nodes[['osmid', 'y', 'x', 'geometry', 'state', 'cluster']]
    non_main_road_nodes = non_main_road_nodes[['osmid', 'y', 'x', 'geometry', 'state', 'cluster']]
    
    
    # 데이터셋 raw data 저장
    if ~(region_code in os.listdir('./data/result_data/')):
        os.mkdir(f'./data/result_data/{region_code}')
        os.mkdir(f'./data/result_data/{region_code}/raw_data')
        os.mkdir(f'./data/result_data/{region_code}/geojson_data')
    
    
    with open(f'./data/result_data/{region_code}/raw_data/main_nodes.pickle', 'wb') as f:
        pickle.dump(main_road_nodes, f)

    with open(f'./data/result_data/{region_code}/raw_data/main_edges.pickle', 'wb') as f:
        pickle.dump(main_road_edges, f)
        
    with open(f'./data/result_data/{region_code}/raw_data/non_main_nodes.pickle', 'wb') as f:
        pickle.dump(non_main_road_nodes, f)

    with open(f'./data/result_data/{region_code}/raw_data/non_main_edges.pickle', 'wb') as f:
        pickle.dump(non_main_road_edges, f)
        
        
    '''
    ###### cluster data로 변환
    '''
    '''
    # Main road (간선도로)
    '''
    # 간선도로에서 연결되어 있지 않은 도로 제거
    main_count = pd.DataFrame(main_road_edges[['u','v']].values.reshape(-1)).value_counts()
    main_count = pd.DataFrame(main_count, columns=['count']).reset_index()
    main_count.columns = ['osmid', 'count']

    main_count = {row['osmid']:row['count'] for _,row in  main_count.iterrows()}

    main_road_edges = main_road_edges.loc[[False if (main_count[i[0]]==1) & (main_count[i[1]]==1) else True for i in main_road_edges[['u', 'v']].values]]
    main_road_nodes = main_road_nodes.loc[main_road_nodes['osmid'].isin(set(main_road_edges[['u','v']].values.reshape(-1)))]
    
    # 양쪽에 클러스터에 속하는 경우 복제 해주어서, groupby 사용하기 적합하게 데이터 정제
    main_road_edges_list = []

    for _,row in main_road_edges.iterrows():
        if type(row['cluster']) == list:
            for i in row['cluster']:
                edges = copy.deepcopy(row)
                edges['cluster'] = i
                main_road_edges_list.append(edges)    
        else:
            edges = copy.deepcopy(row)
            main_road_edges_list.append(edges) 

    main_road_edges_list = pd.concat(main_road_edges_list,axis=1).T
    
    
    ### 클러스터 재설정
    # main_road 단일로
    cluster_max = max(main_road_edges_list.loc[main_road_edges_list['cluster'] <  10000]['cluster'])
    singleway_count = sum(main_road_edges_list['cluster'] == -1)

    # tunnel 
    tunnel_re_cluster_dict = {i : cluster_max+singleway_count+1+idx  for idx,i in enumerate(set(main_road_edges_list.loc[main_road_edges_list['cluster'] >= 10000]['cluster']))}

    main_road_edges_list.loc[main_road_edges_list['cluster'] == -1, 'cluster'] = list(range(int(cluster_max+1), int(cluster_max  + singleway_count+1)))
    main_road_edges_list['cluster'] = [tunnel_re_cluster_dict[i] if i >= 10000 else i for i in main_road_edges_list['cluster']]
    
    ### cluster group
    ## intersection
    cluster_id_list = []
    length_list = []
    include_node_list = []
    node_count_list = []
    tunnel_check_list = [] # 0, 1
    type_check_list = []   # single_way, intersection, tunnel
    multiline_list = []

    node_check_dict = {row['osmid']:row['state'] for _,row in main_road_nodes.iterrows()}

    for cluster_id, row in main_road_edges_list[['geometry', 'length', 'cluster', 'u', 'v', 'tunnel']].groupby('cluster'):
        
        if len(row) >= 2:
            cluster_id = cluster_id
            length = round(sum(row['length']),2)
            include_node = list(set(row[['u','v']].values.reshape(-1)))
            node_cnt = sum([1 if node_check_dict[i] == 1 else 0 for i in include_node])
            tunnel_check = int(np.any(row['tunnel'].values != 0))
            type_check = 'tunnel' if np.all(row['tunnel'].values != 0) else 'intersection'
            type_check = 'single_way' if (type_check == 'intersection') & (node_cnt == 0) else type_check
            multiline = MultiLineString(row['geometry'].tolist())
            multiline = linemerge(multiline) 
            
        else:
            cluster_id = cluster_id
            length = round(row['length'].astype('float').iloc[0], 2)
            include_node = list(set(row[['u','v']].values.reshape(-1)))
            node_cnt = sum([1 if node_check_dict[i] == 1 else 0 for i in include_node])
            tunnel_check = int(np.any(row['tunnel'].values != 0))
            type_check = 'tunnel' if np.all(row['tunnel'].values != 0) else 'single_way' 
            multiline = row['geometry'].tolist()[0]
            
        cluster_id_list.append(cluster_id)
        length_list.append(length)
        include_node_list.append(include_node)   
        node_count_list.append(node_cnt)
        tunnel_check_list.append(tunnel_check)   
        type_check_list.append(type_check)   
        multiline_list.append(multiline)
        
    main_cluster_data = pd.DataFrame(cluster_id_list, columns=['cluster_id'])
    main_cluster_data['length'] = length_list
    main_cluster_data['node'] = include_node_list
    main_cluster_data['node_cnt'] = node_count_list
    main_cluster_data['tunnel'] = tunnel_check_list
    main_cluster_data['type'] = type_check_list
    main_cluster_data['geometry'] = multiline_list
    
    main_cluster_data = gpd.GeoDataFrame(main_cluster_data, geometry='geometry')

    # 중심점 추가
    main_cluster_data['centroid_lat'] = [i.centroid.y for i in main_cluster_data['geometry']]
    main_cluster_data['centroid_lon'] = [i.centroid.x for i in main_cluster_data['geometry']]

    main_cluster_data = main_cluster_data.reset_index(drop=True)

    point_df = pd.DataFrame([Point(i[1],i[0]) for i in  main_cluster_data[['centroid_lat', 'centroid_lon']].values], columns=['geometry'])
    point_df = gpd.GeoDataFrame(point_df, geometry='geometry')
    point_df = gpd.sjoin(point_df, HJD).sort_index()

    point_df = point_df[['sgg']]

    main_cluster_data['sgg_code'] = point_df['sgg']

    main_cluster_data['cluster'] = main_cluster_data['cluster_id']
    main_cluster_data['cluster_id'] = [str(i[0])  + str(int(i[1])).zfill(5) for i in  main_cluster_data[['sgg_code', 'cluster_id']].values]
    
    # columns 정렬 
    main_cluster = main_cluster_data[['cluster_id', 'length', 'type', 'sgg_code', 'node', 'node_cnt', 'centroid_lat', 'centroid_lon', 'geometry']]
    
    ### 
    ### 단일로만 뽑아서, 20m 버퍼를 주고 겹치는 cluster 동일 cluster로 부여.
    main_cluster_singleway = main_cluster.loc[main_cluster['type'] == 'single_way']
    main_cluster_geometry = main_cluster_singleway['geometry'] 
    main_cluster_singleway['geometry'] = main_cluster_singleway['geometry'].centroid

    main_cluster_singleway = gpd.GeoDataFrame(main_cluster_singleway, geometry='geometry', crs=4326)

    main_cluster_singleway = main_cluster_singleway.to_crs(epsg=5179)
    main_cluster_singleway.geometry = main_cluster_singleway.geometry.buffer(20)
    main_cluster_singleway = main_cluster_singleway.to_crs(epsg=4326)
    
    #
    test_1 = main_cluster_singleway[['cluster_id', 'geometry']]
    test_2 = main_cluster_singleway[['cluster_id', 'geometry']]

    test = gpd.sjoin(test_1, test_2)
    
    # 
    singleway_re_matching = dict()

    for i in test[['cluster_id_left', 'cluster_id_right']].values:
        try:
            singleway_re_matching[i[0]].append(i[1])
        except:
            singleway_re_matching[i[0]] = [i[1]] 
            

    main_cluster_singleway['re_cluster'] = [singleway_re_matching[i] for i in main_cluster_singleway['cluster_id']]


    for i in main_cluster_singleway[['cluster_id','re_cluster']].values:
        try:
            main_cluster_singleway.loc[main_cluster_singleway['cluster_id'].isin(i[1]), 'cluster_id'] = i[0]
        except:
            pass
        
    main_cluster_singleway['geometry'] = main_cluster_geometry

    cluster_id_list = []
    length_list = []
    include_node_list = []
    node_count_list = []
    sgg_code_list = []
    type_check_list = []   # single_way, intersection, tunnel
    multiline_list = []

    for id, row in main_cluster_singleway.groupby('cluster_id'):
        
        if len(row) > 1:
            cluster_id = id
            length = round(sum(row['length']),2)
            include_node = list(set(list(itertools.chain(*row['node'].tolist()))))
            node_cnt = sum(row['node_cnt'])
            sgg_code = row['sgg_code'].iloc[0]
            type_check = 'single_way'
            multiline = MultiLineString(list(itertools.chain(*[[j for j in i] if type(i)== MultiLineString else [i] for i in row['geometry']])))
            multiline = linemerge(multiline) 
        else: 
            cluster_id = id
            length = row['length'].iloc[0]
            include_node = row['node'].tolist()
            node_cnt = row['node_cnt'].iloc[0]
            sgg_code = row['sgg_code'].iloc[0]
            type_check = 'single_way'
            multiline = row['geometry'].iloc[0]
        
        
        cluster_id_list.append(cluster_id)
        length_list.append(length)
        include_node_list.append(include_node)   
        node_count_list.append(node_cnt)  
        sgg_code_list.append(sgg_code)
        type_check_list.append(type_check)   
        multiline_list.append(multiline)
        
        
    main_cluster_singleway = pd.DataFrame(cluster_id_list, columns=['cluster_id'])
    main_cluster_singleway['length'] = length_list
    main_cluster_singleway['type'] = type_check_list
    main_cluster_singleway['sgg_code'] = sgg_code_list
    main_cluster_singleway['node'] = include_node_list
    main_cluster_singleway['node_cnt'] = node_count_list
    main_cluster_singleway['geometry'] = multiline_list

    main_cluster_singleway = gpd.GeoDataFrame(main_cluster_singleway, geometry='geometry', crs=4326)

    main_cluster_singleway['centroid_lon'] = main_cluster_singleway.centroid.x
    main_cluster_singleway['centroid_lat'] = main_cluster_singleway.centroid.y

    intersection = main_cluster.loc[main_cluster['type'] != 'single_way']
    main_cluster = pd.concat([intersection, main_cluster_singleway])
    main_cluster = main_cluster.reset_index(drop=True)
    
    main_cluster['node'] = [str(i) for i in main_cluster['node']]
    
    # minor cluster 시작 번호를 주기 위해 cluster max 값 저장
    main_cluster_max = max([int(i[5:]) for i in main_cluster['cluster_id']])
    
    # main road cluster data 저장 
    main_cluster.to_file(f'./data/result_data/{region_code}/geojson_data/{region_code}_main_cluster.geojson', driver='GeoJSON')
    
    
    '''
    # Minor road (집산도로)
    '''
    # 간선도로 사이에 있는 짧은 도로 제거
    non_main_road_edges = non_main_road_edges.loc[non_main_road_edges['M_category'] != 9999]
    
    
    # cluster 단위 데이터 생성
    cluster_id_list = []
    length_list = []
    include_node_list = []
    node_count_list = []
    tunnel_check_list = []
    multiline_list = []

    node_check_dict = {row['osmid']:row['state'] for _,row in non_main_road_nodes.iterrows()}
    
    for cluster_id, row in non_main_road_edges[['geometry', 'length', 'M_category', 'u', 'v', 'tunnel']].groupby('M_category'):
        
        ###
        cluster_id = cluster_id
        length = round(sum(row['length']),2)
        include_node = list(set(row[['u','v']].values.reshape(-1)))
        node_cnt = sum([1 if node_check_dict[i] == 1 else 0 for i in include_node])
        tunnel_check = int(np.any(row['tunnel'].values != 0))
        
        multiline = MultiLineString(row['geometry'].tolist())
        multiline = linemerge(multiline)
        ###
        
        cluster_id_list.append(cluster_id)
        length_list.append(length)
        include_node_list.append(include_node)   
        node_count_list.append(node_cnt)
        tunnel_check_list.append(tunnel_check)
        multiline_list.append(multiline)
        
    non_main_cluster_data = pd.DataFrame(cluster_id_list, columns=['cluster_id'])
    non_main_cluster_data['length'] = length_list
    non_main_cluster_data['tunnel'] = tunnel_check_list
    non_main_cluster_data['node'] = include_node_list
    non_main_cluster_data['node_cnt'] = node_count_list
    non_main_cluster_data['geometry'] = multiline_list

    non_main_cluster_data = gpd.GeoDataFrame(non_main_cluster_data, geometry='geometry')
    
    # 중심점 추가
    non_main_cluster_data['centroid_lat'] = [i.centroid.y for i in non_main_cluster_data['geometry']]
    non_main_cluster_data['centroid_lon'] = [i.centroid.x for i in non_main_cluster_data['geometry']]

    # cluster 1km 이하인 곳 제거
    non_main_cluster_data = non_main_cluster_data.loc[non_main_cluster_data['length'] >= 1000]
    non_main_cluster_data = non_main_cluster_data.reset_index(drop=True)
    
    
    
    point_df = pd.DataFrame([Point(i[1],i[0]) for i in  non_main_cluster_data[['centroid_lat', 'centroid_lon']].values], columns=['geometry'])
    point_df = gpd.GeoDataFrame(point_df, geometry='geometry')
    point_df = gpd.sjoin(point_df, HJD).sort_index()

    point_df = point_df[['sgg']]

    non_main_cluster_data['sgg_code'] = point_df['sgg']

    # 집산도로 클러스터 이름 재정의
    non_main_cluster_dict = {i:main_cluster_max+1+idx for idx,i in enumerate(sorted(set(non_main_cluster_data['cluster_id'])))}
    non_main_cluster_data['cluster_id'] = [non_main_cluster_dict[i] for i in non_main_cluster_data['cluster_id']]

    non_main_cluster_data['cluster_id'] = [str(i[0])  + str(int(i[1])).zfill(5) for i in  non_main_cluster_data[['sgg_code', 'cluster_id']].values]

    # geojson 저장을 위해 node 정보 list 자체를 문자로 변환
    non_main_cluster_data['node'] = [str(i) for i in non_main_cluster_data['node']]
    
    # column 재정렬
    non_main_cluster_data = non_main_cluster_data[['cluster_id', 'length', 'sgg_code', 'node', 'node_cnt', 'centroid_lat', 'centroid_lon', 'geometry']]
    
    non_main_cluster_data.to_file(f'./data/result_data/{region_code}/geojson_data/{region_code}_minor_cluster.geojson', driver='GeoJSON')
    


    ### cluster 시각화
    # 색깔 임의 지정을 위해 랜덤 값 지정
    
    main_road = main_cluster
    minor_road = non_main_cluster_data
    
    
    main_road['r_id'] = np.random.rand(main_road.shape[0]) * 255.0
    main_road['g_id'] = np.random.rand(main_road.shape[0]) * 255.0
    main_road['b_id'] = np.random.rand(main_road.shape[0]) * 255.0


    minor_road['r_id'] = np.random.rand(minor_road.shape[0]) * 255.0
    minor_road['g_id'] = np.random.rand(minor_road.shape[0]) * 255.0
    minor_road['b_id'] = np.random.rand(minor_road.shape[0]) * 255.0
    
    minor_road['type'] = 'minor_road'
    
    try:
        place = f'대한민국 {place}'
        places = ox.geocode_to_gdf([place])
        places = ox.project_gdf(places)
        places_centroid = [places['lat'].iloc[0], places['lon'].iloc[0]]
    except:
        place = ' '.join(place.split(" ")[::-1])
        places = ox.geocode_to_gdf([place])
        places = ox.project_gdf(places)
        places_centroid = [places['lat'].iloc[0], places['lon'].iloc[0]]
    
    # 지도 그리기
    INITIAL_VIEW_STATE = pdk.ViewState(latitude=places_centroid[0], longitude=places_centroid[1], zoom=10, max_zoom=16, pitch=0, bearing=0)

    layer1 = pdk.Layer(
        'GeoJsonLayer',
        main_road,
        id = 'major_road',
        pickable=True,
        auto_highlight=True,
        tooltip=True,
        lineWidthScale=10,
        getLineWidth=5,
        getLineColor = '[r_id, g_id, b_id]',
        extruded=True,                 
        coverage=1)

    layer2 = pdk.Layer(
        'GeoJsonLayer',
        minor_road,
        id = 'minor_road',
        pickable=True,
        auto_highlight=True,
        tooltip=True,
        lineWidthScale=10,
        getLineWidth=1,
        getLineColor = '[r_id, g_id, b_id]',
        extruded=True,                 
        coverage=1)
    
    base_map = pdk.Deck(layers=[layer1,layer2], 
                        tooltip={
                            "html": "<b>Cluster ID:</b> {cluster_id} <br/> <b>Number of nodes:</b> {node_cnt} <br/> <b>Type:</b> {type}"
                            },
                        initial_view_state=INITIAL_VIEW_STATE)
    
    
    if not('html_data' in os.listdir(f'./data/result_data/{region_code}/')):
        os.mkdir(f"./data/result_data/{region_code}/html_data/")    
    
    base_map.to_html(f'./data/result_data/{region_code}/html_data/{region_code}_cluster_map.html')    
    print('-' * 20)

In [1]:
import os 

In [8]:
if not('30000' in os.listdir('./data/train_prepared_data/')):
    